In [11]:
import os
import lusid
import datetime
import pandas as pd
import lusid.models as models
from lusidtools.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import create_scope_id, identify_cash_items
from lusid.utilities import ApiClientFactory
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response
)
from lusidjam import RefreshingToken

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Define our scope and portfolio code
scope = "As_at_DEMO-"+ create_scope_id()
code = "PORT_0001"
print(f"Scope: {scope}")
print(f"Code: {code}")

LUSID Environment Initialised
LUSID version :  0.6.9517.0
Scope: As_at_DEMO-3ad8-26a5-77fa-0c
Code: PORT_0001


In [12]:
mapping = {
    "instruments" : {
        "identifier_mapping": {
            "ClientInternal": "Internal_id",
        },
        "required": {
            "name": "instr_id"
        },
    },
    "portfolios": {
        "required": {
            "code": "Portfolio",
            "display_name":  "$As-at_POC",
            "base_currency": "$GBP",
            "created": "$2018-01-01T00:00:00+00:00"
        },
    }
}

In [13]:
# Call LUSID to upsert instruments
response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pd.DataFrame({"instr_id":['GBP'], 'Internal_id':['GBP']}),
        mapping_required=mapping["instruments"]["required"],
        file_type="instruments",
        mapping_optional={},
        identifier_mapping=mapping["instruments"]["identifier_mapping"]
)

# format response object
success, failed, errors = format_instruments_response(response)

pd.DataFrame(data=[{"success": len(success), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


In [16]:
# call LUSID to create a portfolio or update details of an existing portfolio
response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pd.DataFrame({'Portfolio':['PORT_0001']}),
        mapping_required=mapping["portfolios"]["required"],
        file_type="portfolios",
        mapping_optional={},
)
# format response object
success, errors = format_portfolios_response(response)

pd.DataFrame(data=[{"success": len(success), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


In [17]:


holding_adjustment = [
        models.AdjustHoldingRequest(
            instrument_identifiers={
                'Instrument/default/Currency': 'GBP'},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=999,
                    cost=models.CurrencyAndAmount(
                        amount=999,
                        currency='GBP'),
                    portfolio_cost=999,
                    price=1)
                    ]
        )
    ]

day1_initial_response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=scope,
        code=code,
        effective_at=datetime.datetime(2022, 3, 3, 12, tzinfo=datetime.timezone.utc),
        adjust_holding_request=holding_adjustment)

In [18]:
holding_adjustment = [
        models.AdjustHoldingRequest(
            instrument_identifiers={
                'Instrument/default/Currency': 'GBP'},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=1000,
                    cost=models.CurrencyAndAmount(
                        amount=1000,
                        currency='GBP'),
                    portfolio_cost=1000,
                    price=1)
                    ]
        )
    ]

day1_amended_response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=scope,
        code=code,
        effective_at=datetime.datetime(2022, 3, 3, 12, 30, tzinfo=datetime.timezone.utc),
        adjust_holding_request=holding_adjustment)

In [19]:
# Define some helper functions
def get_all_historical_holdings(scope, code, as_at = None, effective_at=None):
    #Get the "latest" view?
    return api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings_adjustment(
                scope=scope,
                code=code,
                effective_at=effective_at,
            )


In [20]:
# get holdings for Day 1 as at Day 1
response = get_all_historical_holdings(scope, code, effective_at="2020-01-01T00:00:00.000000+00:00")

display(response)

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 27 Jun 2022 09:09:33 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4994', 'X-Rate-Limit-Reset': '2022-06-27T09:10:20.3442201Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HMIM4IDJ14NO:00000003', 'lusid-meta-correlationId': '0HMIM4IDJ14NO:00000003', 'lusid-meta-duration': '216', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com https://editor.swagger.io; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com https://validator.swagger.io; style-src 'unsafe-inline' 'self' https://*.lusid.